<a href="https://colab.research.google.com/github/Jaronex/metropolis-hastings-projects/blob/main/metropolis_hastings_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook is a code-along to *A Practical Guide to MCMC Part 1: MCMC Basics* by Justin Ellis. The tutorial can be found [here](https://jellis18.github.io/post/2018-01-02-mcmc-part1/). My goal is to understand the underlying mechanisms behind Metropolis-Hastings sampling and eventually integrate it into more sophisticated models e.g. [MHGAN](https://arxiv.org/pdf/1811.11357.pdf), [PMH](https://arxiv.org/pdf/1511.01707.pdf) etc.

MCMCs are used to map out and sample probability distributions/density functions. The following apply:

1.   MCMCs explore parameter space such that histogram of samples produce target distribution
2.   Markovian: Movement from one state to the next only depends on current position and transition probability i.e. conditionally independent of prior states
3.   Chain will converge to the target distribution if transition probability is *irreducible*, *positive recurrent* and *aperiodic* 




## Metropolis-Hastings MCMC

In essence, the Metropolis-Hastings algorithm accepts a "jump" from a point in a parameter space $x_i$ to $x_{i+1}$ with the following probability:

$$ κ(x_{i+1}|x_i) = min(1, \frac{π(x_{i+1})q(x_i|x_{i+1})}{π(x_i)q(x_{i+1}|x_i)}) $$

where the fraction above the Hastings ratio, $H$. $H$ is the ratio of the new posterior distribution and transition probability value to the old posterior distribution and transition probability value. If $H > 1$, jump is accepted.

After initialising parameters $x_0$,the following steps can be taken to implement the algorithm $\forall\;i = {1, 2,...,n}$:


1.   Generate proposed parameters: $x_* \sim q(x_*|x_i)$
2.   Sample from uniform distribution $u \sim U(0,1)$
3.   Compute $H$
4.   If $u < min(1, H)$ then $x_{i+1} = x_*$, else $x_{i+1} = x_i$

In [1]:
import numpy as np

In [2]:
def mh_sampler(x0, lnprob_fn, prop_fn, prop_fn_kwargs={}, iterations=100000):
  """
  simple MH sampler.

  x0: initial array of params
  lnprob_fn: fn to compute log-posterior
  prop_fn: fn to perform jumps
  prop_fn_kwargs: keyword args for proposal fn
  iterations: no. of iterations to run sampler

  returns: (chain, acceptance, lnprob) tuple of param chain, acceptance rate
  and log-posterior chain

  note: using log makes computation faster
  """
  # no. of dimensions = no. of params in x0
  ndim = len(x0)

  # init chain, lnprob and acceptance rate
  chain = np.zeros((iterations, ndim)) # rows = iterations, cols = params
  lnprob = np.zeros(iterations)
  accept_rate = np.zeros(iterations)

  # first samples
  chain[0] = x0 # first row of chain is whatever x0 will be
  lnprob0 = lnprob_fn(x0) # first posterior dist is obtained by inputting first element into lnprob_fn
  lnprob[0] = lnprob0 # first element of posterior dist arr is whatever first posterior dist is

  # start loop
  naccept = 0
  for i in range(1, iterations):
    # propose
    x_star, factor = prop_fn(x0, **prop_fn_kwargs) # factor is new trans prob / first trans prob (proposal ratio)
    # draw random uniform no.
    u = np.random.uniform(0, 1)
    # compute hastings ratio
    lnprob_star = lnprob_fn(x_star)
    H = np.exp(lnprob_star - lnprob0) * factor # exponent of ln(new post dist/first post dist)
    # accept/reject step (update acceptance counter)
    if u < H:
      x0 = x_star # accepted
      lnprob0 = lnprob_star # new posterior dist
      naccept += 1
    # update chain
    chain[i] = x0 # i element in chain takes on x0 value (accepted or otherwise)
    lnprob[i] = lnprob0
    accept_rate[i] = naccept / i  # no. of acceptance / no. of iterations

  return chain, accept_rate, lnprob

We now need a proposal distribution. Besides drawing from a prior distribution (which would be computationally inefficient), a Gaussian proposal with **fixed** standard deviation is the simplest proposal.

Mathematically, it is given by $q(x_*|x_i) = N(x_i, \sigma^2)$.

In [3]:
def gaussian_proposal(x, sigma=0.1): # prop_fn
  """
  gaussian proposal distribution.

  draw new params from dist with mean at current position/state x
  and a given std dev sigma

  proposal is symmetric so ratio of proposal densities is 1.

  x: param array
  sigma: std dev of gaussian distribution. can be scalar or vector of length(x)
  returns: new params, ratio of proposal densities

  """

  # draw x_star
  x_star = x + np.random.randn(len(x)) * sigma 
  # proposal ratio is 1 since jump is symmetric
  qxx = 1

  return (x_star, qxx)

We now test the sampler and proposal distribution on a simple 1D Gaussian likelihood function with unknown mean and unit variance. We will use a uniform prior on the mean $\mu ∼ U(-10,10)$.

An explanation on continuous uniform distributions can be found [here](https://en.wikipedia.org/wiki/Continuous_uniform_distribution). Essentially, all support values in a distribution have the same probability density.

In [4]:
def simple_gaussian_lnpost(x): # lnprob_fn, this is NOT the "u" in the acceptance function
  # posterior = prior * likelihood
  # derivation can be found: https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf
  """
  1-D gaussian distribution of means with mu = 0 and std dev = 1.

  prior on mean is U(-10, 10)

  x: param array
  returns: log post
  """
  mu, std = 0, 1
  if -10 < x < 10: # if within supports
    return -0.5 * (x-mu)**2 / std**2 # apply ln to gaussian posterior dist
  else:
    return -1e6 # outside of support, posterior dist becomes negative